# Part 1: Data Extraction

<img src="https://assets-global.website-files.com/6130fa1501794e37c21867cf/63f8f784316c741548e3c0d9_a16z%20webinar.png" width="800" height="800">

# Import Dependencies

In [1]:
import json
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Read CSV

In [2]:
flats = pd.read_csv("hdb_resale_2012_onwards.csv")
flats.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


# Overview of Data Extraction 

<hr>

Based on numerous online articles and past literatures on public housing price prediction in Singapore, it is a well-known fact that HDB prices in Singapore are influenced by their nearby amenities. However, all these information are not available in the `hdb_resale.csv` from the `data.gov.sg`. Hence, in order to enrich the existing dataset we have, we will first create a list of important amenities my group has identified as follow:
1) **MRT/LRT stations**
2) **Bus stops**
3) **Shopping malls**
4) **Educational Institutions**
5) **Nature reserves/parks/green areas**

We will then make use of the `OneMap API` to retrieve their respective latitude and longtitude and the `GeoPy` library find the their distances to each HSB resale flats. Finally, we will add the name of the nearest amenity as well as their distance for each category to the hdb resale dataset. The entire process is done from scratch and documented below.

## 1) Retrive list of amenities
The names of all current amenities in the 5 categories above are extracted from wikipedia and transformed into a python list using `perplexity.ai` to simplify the manual work. All the amenities are current as of 2024.

In [108]:
list_of_mrt = [
    'DHOBY GHAUT (CC1)',
    'BRAS BASAH (CC2)',
    'ESPLANADE (CC3)',
    'PROMENADE (CC4)',
    'NICOLL HIGHWAY (CC5)',
    'STADIUM (CC6)',
    'MOUNTBATTEN (CC7)',
    'DAKOTA (CC8)',
    'PAYA LEBAR (CC9)',
    'MACPHERSON (CC10)',
    'TAI SENG (CC11)',
    'BARTLEY (CC12)',
    'SERANGOON (CC13)',
    'LORONG CHUAN (CC14)',
    'BISHAN (CC15)',
    'MARYMOUNT (CC16)',
    'CALDECOTT (CC17)',
    'BOTANIC GARDENS (CC19)',
    'FARRER ROAD (CC20)',
    'HOLLAND VILLAGE (CC21)',
    'BUONA VISTA (CC22)',
    'ONE-NORTH (CC23)',
    'KENT RIDGE (CC24)',
    'HAW PAR VILLA (CC25)',
    'PASIR PANJANG (CC26)',
    'LABRADOR PARK (CC27)',
    'TELOK BLANGAH (CC28)',
    'HARBOURFRONT (CC29)',
    'PASIR RIS (EW1)',
    'TAMPINES (EW2)',
    'SIMEI (EW3)',
    'TANAH MERAH (EW4)',
    'BEDOK (EW5)',
    'KEMBANGAN (EW6)',
    'EUNOS (EW7)',
    'PAYA LEBAR (EW8)',
    'ALJUNIED (EW9)',
    'KALLANG (EW10)',
    'LAVENDER (EW11)',
    'BUGIS (EW12)',
    'CITY HALL (EW13)',
    'RAFFLES PLACE (EW14)',
    'TANJONG PAGAR (EW15)',
    'OUTRAM PARK (EW16)',
    'TIONG BAHRU (EW17)',
    'REDHILL (EW18)',
    'QUEENSTOWN (EW19)',
    'COMMONWEALTH (EW20)',
    'BUONA VISTA (EW21)',
    'DOVER (EW22)',
    'CLEMENTI (EW23)',
    'JURONG EAST (EW24)',
    'CHINESE GARDEN (EW25)',
    'LAKESIDE (EW26)',
    'BOON LAY (EW27)',
    'PIONEER (EW28)',
    'JOO KOON (EW29)',
    'GUL CIRCLE (EW30)',
    'TUAS CRESCENT (EW31)',
    'TUAS WEST ROAD (EW32)',
    'TUAS LINK (EW33)',
    'JURONG EAST (NS1)',
    'BUKIT BATOK (NS2)',
    'BUKIT GOMBAK (NS3)',
    'CHOA CHU KANG (NS4)',
    'YEW TEE (NS5)',
    'KRANJI (NS7)',
    'MARSILING (NS8)',
    'WOODLANDS (NS9)',
    'ADMIRALTY (NS10)',
    'SEMBAWANG (NS11)',
    'CANBERRA (NS12)',
    'YISHUN (NS13)',
    'KHATIB (NS14)',
    'YIO CHU KANG (NS15)',
    'ANG MO KIO (NS16)',
    'BISHAN (NS17)',
    'BRADDELL (NS18)',
    'TOA PAYOH (NS19)',
    'NOVENA (NS20)',
    'NEWTON (NS21)',
    'ORCHARD (NS22)',
    'SOMERSET (NS23)',
    'DHOBY GHAUT (NS24)',
    'CITY HALL (NS25)',
    'RAFFLES PLACE (NS26)',
    'MARINA BAY (NS27)',
    'MARINA SOUTH PIER (NS28)',
    'BUKIT PANJANG (DT1)',
    'CASHEW (DT2)',
    'HILLVIEW (DT3)',
    'BEAUTY WORLD (DT5)',
    'KING ALBERT PARK (DT6)',
    'SIXTH AVENUE (DT7)',
    'TAN KAH KEE (DT8)',
    'BOTANIC GARDENS (DT9)',
    'STEVENS (DT10)',
    'NEWTON (DT11)',
    'LITTLE INDIA (DT12)',
    'ROCHOR (DT13)',
    'BUGIS (DT14)',
    'PROMENADE (DT15)',
    'BAYFRONT (DT16)',
    'DOWNTOWN (DT17)',
    'TELOK AYER (DT18)',
    'CHINATOWN (DT19)',
    'FORT CANNING (DT20)',
    'BENCOOLEN (DT21)',
    'JALAN BESAR (DT22)',
    'BENDEMEER (DT23)',
    'GEYLANG BAHRU (DT24)',
    'MATTAR (DT25)',
    'MACPHERSON (DT26)',
    'UBI (DT27)',
    'KAKI BUKIT (DT28)',
    'BEDOK NORTH (DT29)',
    'BEDOK RESERVOIR (DT30)',
    'TAMPINES WEST (DT31)',
    'TAMPINES (DT32)',
    'TAMPINES EAST (DT33)',
    'UPPER CHANGI (DT34)',
    'EXPO (DT35)',
    'HARBOURFRONT (NE1)',
    'OUTRAM PARK (NE3)',
    'CHINATOWN (NE4)',
    'CLARKE QUAY (NE5)',
    'DHOBY GHAUT (NE6)',
    'LITTLE INDIA (NE7)',
    'FARRER PARK (NE8)',
    'BOON KENG (NE9)',
    'POTONG PASIR (NE10)',
    'WOODLEIGH (NE11)',
    'SERANGOON (NE12)',
    'KOVAN (NE13)',
    'HOUGANG (NE14)',
    'BUANGKOK (NE15)',
    'SENGKANG (NE16)',
    'PUNGGOL (NE17)',
    'WOODLANDS NORTH (TE1)',
    'WOODLANDS (TE2)',
    'WOODLANDS SOUTH (TE3)',
    'SPRINGLEAF (TE4)',
    'LENTOR (TE5)',
    'MAYFLOWER (TE6)',
    'BRIGHT HILL (TE7)',
    'UPPER THOMSON (TE8)',
    'CALDECOTT (TE9)',
    'STEVENS (TE11)',
    'NAPIER (TE12)',
    'ORCHARD BOULEVARD (TE13)',
    'ORCHARD (TE14)',
    'GREAT WORLD (TE15)',
    'HAVELOCK (TE16)',
    'OUTRAM PARK (TE17)',
    'MAXWELL (TE18)',
    'SHENTON WAY (TE19)',
    'MARINA BAY (TE20)',
    'GARDENS BY THE BAY (TE22)',
    'CHOA CHU KANG (BP1)',
    'SOUTH VIEW (BP2)',
    'KEAT HONG (BP3)',
    'TECK WHYE (BP4)',
    'PHOENIX (BP5)',
    'BUKIT PANJANG (BP6)',
    'PETIR (BP7)',
    'PENDING (BP8)',
    'BANGKIT (BP9)',
    'FAJAR (BP10)',
    'SEGAR (BP11)',
    'JELAPANG (BP12)',
    'SENJA (BP13)',
    'SENGKANG (STC)',
    'CHENG LIM (SW1)',
    'FARMWAY (SW2)',
    'KUPANG (SW3)',
    'THANGGAM (SW4)',
    'FERNVALE (SW5)',
    'LAYAR (SW6)',
    'TONGKANG (SW7)',
    'RENJONG (SW8)',
    'COMPASSVALE (SE1)',
    'RUMBIA (SE2)',
    'BAKAU (SE3)',
    'KANGKAR (SE4)',
    'RANGGUNG (SE5)',
    'PUNGGOL (PTC)',
    'SAM KEE (PW1)',
    'PUNGGOL POINT (PW3)',
    'SAMUDERA (PW4)',
    'NIBONG (PW5)',
    'SUMANG (PW6)',
    'SOO TECK (PW7)',
    'COVE (PE1)',
    'MERIDIAN (PE2)',
    'CORAL EDGE (PE3)',
    'RIVIERA (PE4)',
    'KADALOOR (PE5)',
    'OASIS (PE6)',
    'DAMAI (PE7)'
]

list_of_malls = [
    '100 AM', 
    '600 @ Toa Payoh',
    'Anchorpoint',
    'Beauty World Centre',
    'Beauty World Plaza',
    'Causeway Point',
    'Century Square',
    'CIMB Plaza',
    'City Square Mall',
    'City Vibe',
    'CityLink Mall',
    'Eastpoint Mall',
    'ERA APAC Centre',
    'FairPrice Hub',
    'Forum The Shopping Mall',
    'Fu Lu Shou Complex',
    'Holland Road Shopping Centre',
    'Holland Piazza',
    'Hougang 1',
    'Hougang Mall',
    'IMM',
    'Jem',
    'KINEX',
    'Leisure Park Kallang',
    'Lot One',
    'Marina Bay Link Mall',
    'Mustafa Centre',
    'myVillage @ Serangoon',
    'Nex',
    'Orchard Central',
    'Plaza Singapura',
    'Queensway Shopping Centre',
    'The Rail Mall',
    'Rivervale Mall',
    'Rochester Mall',
    'Seletar Mall',
    'Sembawang Shopping Centre',
    'Sim Lim Square',
    'Sim Lim Tower',
    'The Centrepoint',
    'The Majestic',
    'The Star Vista',
    'Tampines 1',
    'Tampines Mall',
    'Tekka Centre',
    'Thomson Plaza',
    'Upper Serangoon Shopping Centre',
    'VivoCity',
    'West Mall',
    'White Sands',
    'Wisma Geylang Serai',
    'Yew Tee Point',
    '313@Somerset',
    'AMK Hub',
    'Alexandra Retail Centre',
    'Bedok Mall',
    'Bugis Junction',
    'Bugis+',
    'Bukit Panjang Plaza',
    'Capitol Singapore',
    'Changi City Point',
    'Chinatown Point',
    'Clarke Quay Central',
    'Claymore Connect',
    'Clementi Mall',
    'Compass One',
    'Djitsun Mall',
    'Esplanade Mall',
    'Far East Plaza',
    'Funan',
    'Golden Mile Complex',
    'Great World',
    'HarbourFront Centre',
    'Hillion Mall',
    'i12 Katong',
    'ION Orchard',
    'International Plaza',
    'Jewel Changi Airport',
    'Junction 8',
    'Junction 10',
    'Junction Nine',
    'Jurong Point',
    'Kallang Wave Mall',
    'Katong Shopping Centre',
    'Lucky Plaza',
    'Mandarin Gallery',
    'The Shoppes at Marina Bay Sands',
    'Marina Square',
    'Millenia Walk',
    'Ngee Ann City',
    'Northpoint City',
    'Novena Square',
    'One Raffles Place',
    'Orchard Gateway',
    'Orchard Towers',
    'PLQ Mall',
    'Palais Renaissance',
    'Parkway Parade',
    'Paya Lebar Square',
    'Peninsula Plaza',
    "People's Park Centre",
    "People's Park Complex",
    'Raffles City',
    'Sengkang Grand Mall',
    'Shaw House and Centre',
    'SingPost Centre',
    'South Beach Square 2',
    'Sun Plaza',
    'Suntec City Mall',
    'Tang Plaza',
    'Tekka Place',
    'The Paragon',
    'The Woodleigh Mall',
    'Tiong Bahru Plaza',
    'United Square',
    'Valley Point',
    'Waterway Point',
    'West Coast Plaza',
    'Westgate',
    'Wheelock Place',
    'Wisma Atria',
    'Wisteria Mall',
    '888 Plaza',
    'Admiralty Place',
    'Buangkok Square',
    'Canberra Plaza',
    'Dawson Place',
    'Depot Heights Shopping Centre',
    'Elias Mall',
    'Fajar Shopping Centre',
    'Gek Poh Shopping Centre',
    'Greenridge Shopping Centre',
    'Heartland Mall',
    'HDB Hub',
    'Hougang Rivercourt',
    'Limbang Shopping Centre',
    'Loyang Point',
    'Northshore Plaza',
    'Oasis Terraces',
    'Our Tampines Hub',
    'Pasir Ris West Plaza',
    'Pioneer Mall',
    'Punggol Plaza',
    'Rivervale Plaza',
    'Sunshine Place',
    'Taman Jurong Shopping Centre',
    'Vista Point',
    'Woodlands North Plaza',
    'Yew Tee Square',
    '321 Clementi',
    'Cathay Cineleisure Orchard',
    'GV Yishun',
    'The Cathay'
]

list_of_npc = [
    'Bukit Merah East Neighbourhood Police Centre', 
    'Marina Bay Neighbourhood Police Centre', 
    'Rochor Neighbourhood Police Centre',
    'Bukit Merah West Neighbourhood Police Centre', 
    'Clementi Neighbourhood Police Centre', 
    'Jurong East Neighbourhood Police Centre',
    'Queenstown Neighbourhood Police Centre', 
    'Bishan Neighbourhood Police Centre', 
    'Bukit Timah Neighbourhood Police Centre',
    'Kampong Java Neighbourhood Police Centre', 
    'Orchard Neighbourhood Police Centre', 
    'Toa Payoh Neighbourhood Police Centre',
    'Ang Mo Kio North Neighbourhood Police Centre', 
    'Ang Mo Kio South Neighbourhood Police Centre', 
    'Hougang Neighbourhood Police Centre',
    'Punggol Neighbourhood Police Centre', 
    'Sembawang Neighbourhood Police Centre', 
    'Sengkang Neighbourhood Police Centre', 
    'Serangoon Neighbourhood Police Centre',
    'Yishun North Neighbourhood Police Centre', 
    'Yishun South Neighbourhood Police Centre', 
    'Bedok Neighbourhood Police Centre',
    'Changi Neighbourhood Police Centre',
    'Geylang Neighbourhood Police Centre',
    'Marine Parade Neighbourhood Police Centre', 
    'Pasir Ris Neighbourhood Police Centre',
    'Tampines Neighbourhood Police Centre', 
    'Bukit Batok Neighbourhood Police Centre', 
    'Bukit Panjang Neighbourhood Police Centre',
    'Choa Chu Kang Neighbourhood Police Centre', 
    'Jurong West Neighbourhood Police Centre', 
    'Nanyang Neighbourhood Police Centre',
    'Woodlands Neighbourhood Police Centre', 
    'Woodlands East Neighbourhood Police Centre', 
    'Woodlands West Neighbourhood Police Centre'
]

singapore_primary_schools = [
    "Admiralty Primary School",
    "Ahmad Ibrahim Primary School",
    "Ai Tong School",
    "Alexandra Primary School",
    "Anchor Green Primary School",
    "Anderson Primary School",
    "Ang Mo Kio Primary School",
    "Anglo-Chinese School (Junior)",
    "Anglo-Chinese School (Primary)",
    "Angsana Primary School",
    "Beacon Primary School",
    "Bedok Green Primary School",
    "Bendemeer Primary School",
    "Blangah Rise Primary School",
    "Boon Lay Garden Primary School",
    "Bukit Panjang Primary School",
    "Bukit Timah Primary School",
    "Bukit View Primary School",
    "Canberra Primary School",
    "Canossa Catholic Primary School",
    "Cantonment Primary School",
    "Casuarina Primary School",
    "Catholic High School",
    "Cedar Primary School",
    "Changkat Primary School",
    "CHIJ (Katong) Primary",
    "CHIJ (Kellock)",
    "CHIJ Our Lady of Good Counsel",
    "CHIJ Our Lady of the Nativity",
    "CHIJ Our Lady Queen of Peace",
    "CHIJ Primary (Toa Payoh)",
    "CHIJ St. Nicholas Girls’ School",
    "Chongfu School",
    "Chongzheng Primary School",
    "Chua Chu Kang Primary School",
    "Clementi Primary School",
    "Compassvale Primary School",
    "Concord Primary School",
    "Corporation Primary School",
    "Damai Primary School",
    "Dazhong Primary School",
    "De La Salle School",
    "East Spring Primary School",
    "Edgefield Primary School",
    "Elias Park Primary School",
    "Endeavour Primary School",
    "Eunos Primary School",
    "Evergreen Primary School",
    "Fairfield Methodist School (Primary)",
    "Farrer Park Primary School",
    "Fengshan Primary School",
    "Fern Green Primary School",
    "Fernvale Primary School",
    "First Toa Payoh Primary School",
    "Frontier Primary School",
    "Fuchun Primary School",
    "Fuhua Primary School",
    "Gan Eng Seng Primary School",
    "Geylang Methodist School (Primary)",
    "Gongshang Primary School",
    "Greendale Primary School",
    "Greenridge Primary School",
    "Greenwood Primary School",
    "Guangyang Primary School",
    "Haig Girls’ School",
    "Henry Park Primary School",
    "Holy Innocents’ Primary School",
    "Hong Wen School",
    "Horizon Primary School",
    "Hougang Primary School",
    "Huamin Primary School",
    "Innova Primary School",
    "Jiemin Primary School",
    "Jing Shan Primary School",
    "Junyuan Primary School",
    "Jurong Primary School",
    "Jurong West Primary School",
    "Juying Primary School",
    "Keming Primary School",
    "Kheng Cheng School",
    "Kong Hwa School",
    "Kranji Primary School",
    "Kuo Chuan Presbyterian Primary School",
    "Lakeside Primary School",
    "Lianhua Primary School",
    "Maha Bodhi School",
    "Maris Stella High School",
    "Marsiling Primary School",
    "Marymount Convent School",
    "Mayflower Primary School",
    "Mee Toh School",
    "Meridian Primary School",
    "Methodist Girls’ School (Primary)",
    "Montfort Junior School",
    "Nan Chiau Primary School",
    "Nan Hua Primary School",
    "Nanyang Primary School",
    "Naval Base Primary School",
    "New Town Primary School",
    "Ngee Ann Primary School",
    "North Spring Primary School",
    "North View Primary School",
    "North Vista Primary School",
    "Northland Primary School",
    "Northoaks Primary School",
    "Oasis Primary School",
    "Palm View Primary School",
    "Park View Primary School",
    "Pasir Ris Primary School",
    "Paya Lebar Methodist Girls' School (Primary)",
    "Pei Chun Public School",
    "Pei Hwa Presbyterian Primary School",
    "Pei Tong Primary School",
    "Peiying Primary School",
    "Pioneer Primary School",
    "Poi Ching School",
    "Princess Elizabeth Primary School",
    "Punggol Cove Primary School",
    "Punggol Green Primary School",
    "Punggol Primary School",
    "Punggol View Primary School",
    "Qihua Primary School",
    "Queenstown Primary School",
    "Radin Mas Primary School",
    "Raffles Girls' Primary School",
    "Red Swastika School",
    "River Valley Primary School",
    "Riverside Primary School",
    "Rivervale Primary School",
    "Rosyth School",
    "Rulang Primary School",
    "Sembawang Primary School",
    "Seng Kang Primary School",
    "Shuqun Primary School",
    "Si Ling Primary School",
    "Singapore Chinese Girls' Primary School",
    "South View Primary School",
    "Springdale Primary School",
    "St. Andrew's Junior School",
    "St. Anthony's Canossian Primary School",
    "St. Anthony's Primary School",
    "St. Gabriel's Primary School",
    "St. Hilda's Primary School",
    "St. Joseph's Institution Junior",
    "St. Margaret's Primary School",
    "St. Stephen's School",
    "Stamford Primary School",
    "Tampines North Primary School",
    "Tampines Primary School",
    "Tanjong Katong Primary School",
    "Tao Nan School",
    "Teck Ghee Primary School",
    "Teck Whye Primary School",
    "Telok Kurau Primary School",
    "Temasek Primary School",
    "Townsville Primary School",
    "Unity Primary School",
    "Valour Primary School",
    "Waterway Primary School",
    "Wellington Primary School",
    "West Grove Primary School",
    "West Spring Primary School",
    "West View Primary School",
    "Westwood Primary School",
    "White Sands Primary School",
    "Woodgrove Primary School",
    "Woodlands Primary School",
    "Woodlands Ring Primary School",
    "Xinghua Primary School",
    "Xingnan Primary School",
    "Xinmin Primary School",
    "Xishan Primary School",
    "Yangzheng Primary School",
    "Yew Tee Primary School",
    "Yio Chu Kang Primary School",
    "Yishun Primary School",
    "Yu Neng Primary School",
    "Yuhua Primary School",
    "Yumin Primary School",
    "Zhangde Primary School",
    "Zhenghua Primary School",
    "Zhonghua Primary School"
]

singapore_secondary_schools = [
    "Ahmad Ibrahim Secondary School",
    "Anderson Secondary School",
    "Ang Mo Kio Secondary School",
    "Anglican High School, Singapore",
    "Anglo-Chinese School (Barker Road)",
    "Anglo-Chinese School (Independent)",
    "Bartley Secondary School",
    "Beatty Secondary School",
    "Bedok Green Secondary School",
    "Bedok South Secondary School",
    "Bedok View Secondary School",
    "Boon Lay Secondary School",
    "Bowen Secondary School",
    "Braddell Secondary School",
    "Broadrick Secondary School",
    "Bukit Batok Secondary School",
    "Bukit Merah Secondary School",
    "Bukit Panjang Government High School",
    "Bukit View Secondary School",
    "Canberra Secondary School",
    "Catholic High School",
    "Cedar Girls' Secondary School",
    "CHIJ Katong Convent",
    "CHIJ Saint Joseph's Convent",
    "CHIJ Saint Nicholas Girls' School",
    "CHIJ Saint Theresa's Convent",
    "CHIJ Secondary (Toa Payoh)",
    "The Chinese High School",
    "Christ Church Secondary School",
    "Chua Chu Kang Secondary School",
    "Chung Cheng High School (Main)",
    "Chung Cheng High School (Yishun)",
    "Clementi Town Secondary School",
    "Commonwealth Secondary School",
    "Compassvale Secondary School",
    "Crescent Girls' School",
    "Crest Secondary School",
    "Damai Secondary School",
    "Deyi Secondary School",
    "Dunman High School",
    "Dunman Secondary School",
    "East Spring Secondary School",
    "Edgefield Secondary School",
    "Evergreen Secondary School",
    "Fairfield Methodist School (Secondary)",
    "Fajar Secondary School",
    "Fuchun Secondary School",
    "Fuhua Secondary School",
    "Gan Eng Seng School",
    "Geylang Methodist School (Secondary)",
    "Greendale Secondary School",
    "Greenridge Secondary School",
    "Guangyang Secondary School",
    "Hai Sing Catholic School",
    "Hillgrove Secondary School",
    "Holy Innocents' High School",
    "Hougang Secondary School",
    "Hua Yi Secondary School",
    "Hwa Chong Institution",
    "Junyuan Secondary School",
    "Jurong Secondary School",
    "Jurongville Secondary School",
    "Kranji Secondary School",
    "Kuo Chuan Presbyterian Secondary School",
    "Loyang View Secondary School",
    "Madrasah Al-Arabiah Al-Islamiah",
    "Madrasah Al-Maarif Al-Islamiah",
    "Madrasah Aljunied Al-Islamiah",
    "Manjusri Secondary School",
    "Maris Stella High School",
    "Marsiling Secondary School",
    "Mayflower Secondary School",
    "Meridian Secondary School",
    "Methodist Girls' School",
    "Montfort Secondary School",
    "Nan Chiau High School",
    "Nan Hua High School",
    "Nanyang Girls' High School",
    "Naval Base Secondary School",
    "New Town Secondary School",
    "Ngee Ann Secondary School",
    "North View Secondary School",
    "Northbrooks Secondary School",
    "NUS High School of Math and Science",
    "Orchid Park Secondary School",
    "Outram Secondary School",
    "Pasir Ris Secondary School",
    "Paya Lebar Methodist Girls' School (Secondary)",
    "Pei Hwa Secondary School",
    "Peirce Secondary School",
    "Pioneer Secondary School",
    "Presbyterian High School",
    "Queenstown Secondary School",
    "Queensway Secondary School",
    "Raffles Girls' School (Secondary)",
    "Raffles Institution",
    "River Valley High School",
    "Riverside Secondary School",
    "Saint Andrew's Secondary School",
    "Saint Anthony's Canossian Secondary School",
    "St. Gabriel's Secondary School",
    "St. Hilda's Secondary School",
    "Saint Joseph's Institution",
    "Saint Margaret's Secondary School",
    "Saint Patrick's School",
    "San Yu Adventist School",
    "School of Science and Technology",
    "Sembawang Secondary School",
    "Sengkang Secondary School",
    "Serangoon Garden Secondary School",
    "Serangoon Secondary School",
    "Singapore Chinese Girls' School",
    "Singapore Sports School",
    "SJI International School",
    "Springfield Secondary School",
    "Swiss Cottage Secondary School",
    "Tampines Secondary School",
    "Tanglin Secondary School",
    "Tanjong Katong Girls' School",
    "Tanjong Katong Secondary School",
    "Teck Whye Secondary School",
    "Temasek Secondary School",
    "Tiong Bahru Secondary School",
    "Victoria School",
    "West Spring Secondary School",
    "Westwood Secondary School",
    "Whampoa Secondary School",
    "Whitley Secondary School",
    "Xinmin Secondary School",
    "Yio Chu Kang Secondary School",
    "Yishun Secondary School",
    "Yishun Town Secondary School",
    "Yuan Ching Secondary School",
    "Yuhua Secondary School",
    "Yusof Ishak Secondary School",
    "Yuying Secondary School",
    "Zhenghua Secondary School",
    "Zhonghua Secondary School"
]

tertiary_education_institutions = [
    # Junior Colleges
    "Anderson Serangoon Junior College",
    "Anglo-Chinese Junior College",
    "Catholic Junior College",
    "Dunman High School",
    "Eunoia Junior College",
    "Hwa Chong Institution",
    "Jurong Pioneer Junior College",
    "Millennia Institute",
    "Nanyang Junior College",
    "National Junior College",
    "NUS High School of Mathematics and Science",
    "Raffles Institution",
    "River Valley High School",
    "School of the Arts",
    "Singapore Sports School",
    "Saint Andrew’s Junior College",
    "Saint Joseph’s Institution",
    "Tampines Meridian Junior College",
    "Temasek Junior College",
    "Victoria Junior College",
    "Yishun Innova Junior College",
    "Nanyang Polytechnic",
    "Ngee Ann Polytechnic",
    "Republic Polytechnic",
    "Singapore Polytechnic",
    "Temasek Polytechnic"
]

singapore_nature_reserves_areas_parks = [
        "Bukit Timah Nature Reserve",
        "Central Catchment Nature Reserve",
        "Sungei Buloh Wetland Reserve",
        "Labrador Nature Reserve",
        "Kent Ridge Park",
        "Kranji Coastal Nature Park",
        "Mount Faber Park",
        "Sungei Cina",
        "Telok Blangah Hill Park",
        "Admiralty Park",
        "Ang Mo Kio Town Garden East",
        "Ang Mo Kio Town Garden West",
        "Bedok Town Park",
        "Bishan-Ang Mo Kio Park",
        "Bukit Batok Nature Park",
        "Bukit Batok Town Park",
        "Changi Beach Park",
        "Choa Chu Kang Park",
        "Clementi Woods Park",
        "Coney Island Park",
        "Dairy Farm Nature Park",
        "Dhoby Ghaut Green",
        "East Coast Park",
        "Esplanade Park",
        "Fort Canning Park",
        "Gardens by the Bay",
        "Hindhede Nature Park",
        "Hong Lim Park",
        "HortPark",
        "Istana Park",
        "Japanese Cemetery Park",
        "Jurong Lake Gardens",
        "Jurong Central Park",
        "Kallang Riverside Park",
        "Kranji Marshes",
        "Kranji Reservoir Park",
        "Lower Seletar Reservoir Park",
        "MacRitchie Reservoir Park",
        "Marsiling Park",
        "Mount Emily Park",
        "One-north Park",
        "Pasir Ris Park",
        "Pasir Ris Town Park",
        "Pearl's Hill City Park",
        "Punggol Park",
        "Punggol Point Park",
        "Punggol Waterway Park",
        "Rifle Range Nature Park",
        "Sembawang Park",
        "Sengkang Riverside Park",
        "Singapore Botanic Gardens",
        "Sun Plaza Park",
        "Tampines Eco Green",
        "Tanjong Pagar Park",
        "The Southern Ridges",
        "Tiong Bahru Park",
        "Toa Payoh Town Park",
        "Upper Peirce Reservoir Park",
        "Upper Seletar Reservoir",
        "War Memorial Park",
        "West Coast Park",
        "Woodlands Waterfront Park"
]

## 2) Retrieve Latitude and Longtitude for all MRT and LRT Stations
The station names and their coordinates are extracted and saved as a csv called `mrt_data.csv`.

In [74]:
# get station_name, lat, long for all MRT and LRT stations

station_name = []
station_lat = []
station_long = []

url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
for search_name in list_of_mrt:
    resp = requests.get(url.format(search_name))
    data = json.loads(resp.content)
    if data["found"] != 0:
        station_name.append(data["results"][0]["BUILDING"])
        station_lat.append(data["results"][0]["LATITUDE"])
        station_long.append(data["results"][0]["LONGITUDE"])
    else:
        station_name.append("Not Found")
        station_lat.append("Not Found")
        station_long.append("Not Found")

mrt_df = pd.DataFrame({
    "Search Name": list_of_mrt,
    "Station Name": station_name,
    "Station Lat": station_lat,
    "Station Long": station_long
})

In [79]:
mrt_df = mrt_df.drop(["Search Name"], axis=1)
mrt_df.to_csv("mrt_data.csv", index=False)

## 3) Retrieve Latitude and Longtitude for all malls
The mall names and their coordinates are extracted and saved as a csv called `mall_data.csv`.

In [105]:
# get mall_name, lat, long for all shopping malls

mall_name = []
mall_lat = []
mall_long = []

url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
for search_name in list_of_malls:
    resp = requests.get(url.format(search_name))
    data = json.loads(resp.content)
    if data["found"] != 0:
        mall_name.append(data["results"][0]["BUILDING"])
        mall_lat.append(data["results"][0]["LATITUDE"])
        mall_long.append(data["results"][0]["LONGITUDE"])
    else:
        mall_name.append("Not Found")
        mall_lat.append("Not Found")
        mall_long.append("Not Found")

mall_df = pd.DataFrame({
    "Search Name": list_of_malls,
    "Mall Name": mall_name,
    "Mall Lat": mall_lat,
    "Mall Long": mall_long
})

In [106]:
mall_df = mall_df.drop(mall_df[mall_df["Mall Name"] == "Not Found"].index)
mall_df["Mall Name"] = mall_df["Mall Name"].str.replace("DBS", "")
mall_df["Mall Name"] = mall_df["Mall Name"].str.replace("OCBC", "")
mall_df["Mall Name"] = mall_df["Mall Name"].str.replace("UOB", "")
mall_df = mall_df.drop(["Search Name"], axis=1)
mall_df.to_csv("mall_data.csv", index=False)

## 4) Retrieve Latitude and Longtitude for all bus stops
It is not practical to extract out the name of all the bus stop from online sources. Hence, I iterate through a range of integers to identify their code instead. The bus stop code and their coordinates are extracted and saved as a csv called `bus_stop_df.csv`.

In [1]:
printbus_stop_code = []
bus_stop_lat = []
bus_stop_long = []

list_of_bus_stops = range(1000,99190)
url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
for code in list_of_bus_stops:
    code_str = str(code) + " (BUS STOP)"
    
    if (code < 10000):
        code_str = "0" + code_str
        
    resp = requests.get(url.format(code_str))
    try:
        data = json.loads(resp.content)
        if data["found"] != 0:
            bus_stop_code.append(data["results"][0]["BUILDING"])
            bus_stop_lat.append(data["results"][0]["LATITUDE"])
            bus_stop_long.append(data["results"][0]["LONGITUDE"])
    except:
        continue

In [2]:
bus_stop_df = pd.DataFrame({
    "Bus Stop": bus_stop_code,
    "Bus Stop Lat": bus_stop_lat,
    "Bus Stop Long": bus_stop_long
})
bus_stop_df.to_csv("bus_stop_df.csv", index=False)

## 5) Retrieve Latitude and Longtitude for all NPC
The NPC names and their coordinates are extracted and saved as a csv called `npc_data.csv`.

In [27]:
# get mall_name, lat, long for all shopping malls

npc_name = []
npc_lat = []
npc_long = []

url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
for search_name in list_of_npc:
    resp = requests.get(url.format(search_name))
    data = json.loads(resp.content)
    if data["found"] != 0:
        npc_name.append(data["results"][0]["BUILDING"])
        npc_lat.append(data["results"][0]["LATITUDE"])
        npc_long.append(data["results"][0]["LONGITUDE"])
    else:
        npc_name.append("Not Found")
        npc_lat.append("Not Found")
        npc_long.append("Not Found")

npc_df = pd.DataFrame({
    "Search Name": list_of_npc,
    "NPC": npc_name,
    "NPC Lat": npc_lat,
    "NPC Long": npc_long
})

In [28]:
npc_df = npc_df.drop("NPC", axis=1)
npc_df = npc_df.rename({"Search Name": "NPC"}, axis=1)
npc_df.to_csv("npc_data.csv", index=False)

## 6) Retrieve Latitude and Longitude for all schools 
The school names and their coordinates are extracted and saved as a csv called `school_data.csv`.

In [113]:
schools = singapore_primary_schools + singapore_secondary_schools + tertiary_education_institutions

school_name = []
school_lat = []
school_long = []

url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
for search_name in schools:
    resp = requests.get(url.format(search_name.upper()))
    data = json.loads(resp.content)
    if data["found"] != 0:
        school_name.append(data["results"][0]["BUILDING"])
        school_lat.append(data["results"][0]["LATITUDE"])
        school_long.append(data["results"][0]["LONGITUDE"])
    else:
        school_name.append("Not Found")
        school_lat.append("Not Found")
        school_long.append("Not Found")

school_df = pd.DataFrame({
    "Search Name": schools,
    "School": school_name,
    "School Lat": school_lat,
    "School Long": school_long
})

In [114]:
school_df = school_df.drop(school_df[school_df["School"] == "Not Found"].index)
school_df = school_df.drop("School", axis=1)
school_df = school_df.rename({"Search Name":"School Name"})

In [115]:
school_df.to_csv("school_data.csv", index=False)

## 7) Retrieve Latitude and Longitude for all green areas
The green area names and their coordinates are extracted and saved as a csv called `green_data.csv`.

In [109]:
green_name = []
green_lat = []
green_long = []

url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
for search_name in singapore_nature_reserves_areas_parks:
    resp = requests.get(url.format(search_name))
    data = json.loads(resp.content)
    if data["found"] != 0:
        green_name.append(data["results"][0]["BUILDING"])
        green_lat.append(data["results"][0]["LATITUDE"])
        green_long.append(data["results"][0]["LONGITUDE"])
    else:
        green_name.append("Not Found")
        green_lat.append("Not Found")
        green_long.append("Not Found")

green_df = pd.DataFrame({
    "Search Name": singapore_nature_reserves_areas_parks,
    "Green": green_name,
    "Green Lat": green_lat,
    "Green Long": green_long
})

In [127]:
green_df = green_df.drop(green_df[green_df["Green"]=="Not Found"].index)
green_df = green_df.drop("Green", axis=1)
green_df = green_df.rename({"Search Name":"Green Area"}, axis=1)

In [112]:
green_df.to_csv("green_data.csv",index=False)

## 8) Retrieve Latitude and Longtitude for all unique addresses in the hdb resale dataset
The HDB address and their coordinates are extracted and saved as a csv called `address_data.csv`.

In [118]:
flats["address"] = flats["block"] + " " + flats["street_name"]
list_of_address = flats["address"].unique().tolist()
print(len(list_of_address))

9624


In [119]:
# get address, lat, long for all hdb resale unit address

address = []
addr_lat = []
addr_long = []

url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
for search_name in list_of_address:
    resp = requests.get(url.format(search_name))
    data = json.loads(resp.content)
    if data["found"] != 0:
        address.append(f"{data['results'][0]['BLK_NO']} {data['results'][0]['ROAD_NAME']}")
        addr_lat.append(data["results"][0]["LATITUDE"])
        addr_long.append(data["results"][0]["LONGITUDE"])
    else:
        address.append("Not Found")
        addr_lat.append("Not Found")
        addr_long.append("Not Found")

addr_df = pd.DataFrame({
    "Search Name": list_of_address,
    "Address": address,
    "Address Lat": addr_lat,
    "Address Long": addr_long
})

In [131]:
addr_df = addr_df.drop(addr_df[addr_df["Address"]=="Not Found"].index)
addr_df.to_csv("address_data.csv", index=False)
addr_df.head()

,Search Name,Address,Address Lat,Address Long
0,406 ANG MO KIO AVE 10,406 ANG MO KIO AVENUE 10,1.36200453938712,103.853879910407
1,108 ANG MO KIO AVE 4,108 ANG MO KIO AVENUE 4,1.37096635222625,103.838201940326
2,602 ANG MO KIO AVE 5,602 ANG MO KIO AVENUE 5,1.38070883044887,103.835368226602
3,465 ANG MO KIO AVE 10,465 ANG MO KIO AVENUE 10,1.3662010408294,103.857200967235
4,601 ANG MO KIO AVE 5,601 ANG MO KIO AVENUE 5,1.38104135523576,103.835131742647


## 9) Find the nearest MRT/LRT station / mall / bus stop / NPC / schools / green area and the distances to each address
For each HDB address, i will iterate through the 5 categories and find the nearest amenity as well as their distances to the HDB unit. However, the distance is only an estimation as it is based on straight line distance.

In [133]:
mrt = pd.read_csv("mrt_data.csv")
mall = pd.read_csv("mall_data.csv")
mall = mall.drop(mall[mall["Mall Name"] == "NIL"].index)
bus = pd.read_csv("bus_stop_df.csv")
npc = pd.read_csv("npc_data.csv")
school = pd.read_csv("school_data.csv")
green = pd.read_csv("green_data.csv")
address = pd.read_csv("address_data.csv")

In [140]:
from geopy.distance import great_circle

d_mrt = []
n_mrt = []
d_mall = []
n_mall = []
d_bus = []
n_bus = []
d_npc = []
n_npc = []
d_school = []
n_school = []
d_green = []
n_green = []
num_bus = []

for addr, addr_lat, addr_long in zip(address["Address"],address["Address Lat"],address["Address Long"]):
    
    d_to_mrt = 99
    n_to_mrt = ""
    d_to_mall = 99
    n_to_mall = ""
    d_to_bus = 99
    n_to_bus = ""
    d_to_npc = 99
    n_to_npc = ""
    d_to_school = 99
    n_to_school = ""
    d_to_green = 99
    n_to_green = ""
    temp = 0
    num_bus_temp = 0
    

    for mrt_name, mrt_lat, mrt_long in zip(mrt["Station Name"], mrt["Station Lat"], mrt["Station Long"]):
        temp = great_circle((addr_lat, addr_long), (mrt_lat, mrt_long)).km
        if d_to_mrt > temp:
            d_to_mrt = temp
            n_to_mrt = mrt_name
            
    for mall_name, mall_lat, mall_long in zip(mall["Mall Name"], mall["Mall Lat"], mall["Mall Long"]):
        temp = great_circle((addr_lat, addr_long), (mall_lat, mall_long)).km
        if d_to_mall > temp:
            d_to_mall = temp  
            n_to_mall = mall_name

    for bus_name, bus_lat, bus_long in zip(bus["Bus Stop Code"], bus["Bus Stop Lat"], bus["Bus Stop Long"]):
        temp = great_circle((addr_lat, addr_long), (bus_lat, bus_long)).km
        if d_to_bus > temp:
            d_to_bus = temp  
            n_to_bus = bus_name
        # if distance to bus stop < 200m
        if temp < 0.2:
            num_bus_temp += 1

    for npc_name, npc_lat, npc_long in zip(npc["NPC"], npc["NPC Lat"], npc["NPC Long"]):
        temp = great_circle((addr_lat, addr_long), (npc_lat, npc_long)).km
        if d_to_npc > temp:
            d_to_npc = temp  
            n_to_npc = npc_name

    for school_name, school_lat, school_long in zip(school["Search Name"], school["School Lat"],school["School Long"]):
        temp = great_circle((addr_lat, addr_long), (school_lat, school_long)).km
        if d_to_school > temp:
            d_to_school = temp  
            n_to_school = school_name

    for green_name, green_lat, green_long in zip(green["Search Name"], green["Green Lat"], green["Green Long"]):
        temp = great_circle((addr_lat, addr_long), (green_lat, green_long)).km
        if d_to_green > temp:
            d_to_green = temp  
            n_to_green = green_name
    
    d_mrt.append(d_to_mrt)
    n_mrt.append(n_to_mrt)
    d_mall.append(d_to_mall)
    n_mall.append(n_to_mall)
    d_bus.append(d_to_bus)
    n_bus.append(n_to_bus)
    num_bus.append(num_bus_temp)
    d_npc.append(d_to_npc)
    n_npc.append(n_to_npc)
    d_school.append(d_to_school)
    n_school.append(n_to_school)
    d_green.append(d_to_green)
    n_green.append(n_to_green)

In [141]:
address_info = address.copy()
address_info["Nearest MRT Station"] = n_mrt
address_info["Nearest MRT Distance"] = d_mrt
address_info["Nearest Mall Name"] = n_mall
address_info["Nearest Mall Distance"] = d_mall 
address_info["Nearest Bus Stop Code"] = n_bus
address_info["Nearest Bus Stop Distance"] = d_bus
address_info["Number of Bus Stops"] = num_bus
address_info["Nearest NPC"] = n_npc
address_info["Nearest NPC Distance"] = d_npc
address_info["Nearest School"] = n_school
address_info["Nearest School Distance"] = d_school
address_info["Nearest Green Area"] = n_green
address_info["Nearest Green Area Distance"] = d_green

address_info

,Search Name,Address,Address Lat,Address Long,Nearest MRT Station,Nearest MRT Distance,Nearest Mall Name,Nearest Mall Distance,Nearest Bus Stop Code,Nearest Bus Stop Distance,Number of Bus Stops,Nearest NPC,Nearest NPC Distance,Nearest School,Nearest School Distance,Nearest Green Area,Nearest Green Area Distance
0,406 ANG MO KIO AVE 10,406 ANG MO KIO AVENUE 10,1.362005,103.853880,ANG MO KIO MRT STATION (NS16),1.003997,CATHAY CINEPLEX AMK HUB,1.000654,54319 (BUS STOP),0.091925,2,Bishan Neighbourhood Police Centre,0.829747,Townsville Primary School,0.219348,Bishan-Ang Mo Kio Park,0.693469
1,108 ANG MO KIO AVE 4,108 ANG MO KIO AVENUE 4,1.370966,103.838202,MAYFLOWER MRT STATION (TE6),0.189875,CATHAY CINEPLEX AMK HUB,1.157763,54189 (BUS STOP),0.166313,3,Ang Mo Kio South Neighbourhood Police Centre,1.506198,Ang Mo Kio Primary School,0.242224,Ang Mo Kio Town Garden West,0.636510
2,602 ANG MO KIO AVE 5,602 ANG MO KIO AVENUE 5,1.380709,103.835368,LENTOR MRT STATION (TE5),0.535117,CATHAY CINEPLEX AMK HUB,1.938852,55121 (BUS STOP),0.123479,2,Ang Mo Kio North Neighbourhood Police Centre,1.182304,Anderson Primary School,0.777627,Ang Mo Kio Town Garden West,1.113168
3,465 ANG MO KIO AVE 10,465 ANG MO KIO AVENUE 10,1.366201,103.857201,ANG MO KIO MRT STATION (NS16),0.945529,MYVILLAGE AT SERANGOON GARDEN,0.892904,54389 (BUS STOP),0.069453,2,Ang Mo Kio South Neighbourhood Police Centre,0.737154,Deyi Secondary School,0.517985,Ang Mo Kio Town Garden East,0.898934
4,601 ANG MO KIO AVE 5,601 ANG MO KIO AVENUE 5,1.381041,103.835132,LENTOR MRT STATION (TE5),0.501151,CATHAY CINEPLEX AMK HUB,1.983038,55121 (BUS STOP),0.155488,3,Ang Mo Kio North Neighbourhood Police Centre,1.192218,Anderson Primary School,0.782772,Ang Mo Kio Town Garden West,1.157278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9617,408 ANG MO KIO AVE 10,408 ANG MO KIO AVENUE 10,1.362091,103.854588,ANG MO KIO MRT STATION (NS16),1.035877,CATHAY CINEPLEX AMK HUB,1.042611,54371 (BUS STOP),0.126004,3,Bishan Neighbourhood Police Centre,0.900912,Townsville Primary School,0.243992,Bishan-Ang Mo Kio Park,0.772717
9618,806 HOUGANG CTRL,806 HOUGANG CENTRAL,1.371078,103.894100,HOUGANG MRT STATION (NE14),0.192626,HOUGANG MALL BRANCH,0.159997,64541 (BUS STOP),0.166461,3,Hougang Neighbourhood Police Centre,1.075150,Holy Innocents' High School,0.260751,Japanese Cemetery Park,2.008616
9619,516 WEST COAST RD,516 WEST COAST ROAD,1.308168,103.760446,CLEMENTI MRT STATION (EW23),0.935594,321 CLEMENTI,0.660787,17071 (BUS STOP),0.083418,4,Clementi Neighbourhood Police Centre,1.530594,Tanglin Secondary School,0.306179,West Coast Park,1.193586
9620,127 BT MERAH LANE 1,127 BUKIT MERAH LANE 1,1.285329,103.803397,QUEENSTOWN MRT STATION (EW19),1.067883,QUEENSWAY SHOPPING CENTRE,0.254564,11511 (BUS STOP),0.119381,2,Queenstown Neighbourhood Police Centre,0.344886,Bukit Merah Secondary School,1.117092,HortPark,0.747883


In [39]:
address_info = address.copy()
address_info["Nearest MRT Station"] = n_mrt
address_info["Nearest MRT Distance"] = d_mrt
address_info["Nearest Mall Name"] = n_mall
address_info["Nearest Mall Distance"] = d_mall 
address_info["Nearest Bus Stop Code"] = n_bus
address_info["Nearest Bus Stop Distance"] = d_bus
address_info["Nearest NPC"] = n_npc
address_info["Nearest NPC Distance"] = d_npc

## 10) Save the final CSV
The full dataset with each address and their extracted features is saved to `address_data_full.csv`.

In [142]:
address_info.to_csv("address_data_full.csv",index=False)